In [1]:
import json
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
import numpy as np
%matplotlib inline

# API Data Collection

## Get # of restaurants for year 2012 - 2016

In [3]:
def get_res(years):
    key='38f084f3e7ae058abf3d8155617fface08b2f507'
    #NAICS2013=7225 means Restaurants and Other Eating Places
    #ESTAB means number of establishments 
    # get='get=ESTAB,GEO_TTL,YEAR&NAICS2012=7225&for=zipcode:78701,78702,78759,76701'
    get='get=ESTAB,GEO_TTL,YEAR&NAICS2012=7225&for=zipcode:*'
    initial_year = years[0]
#     time.sleep(1)
    
    census_url="https://api.census.gov/data/{}/zbp?".format(initial_year)
    final_url=census_url+get+"&key="+key
    results={}
    business_response=requests.get(final_url).json()
    print(business_response[0])
    estab_name='ESTAB_{}'.format(initial_year)
    results[estab_name]=[i[0] for i in business_response]
    results['zipcode']=[i[4] for i in business_response] 
    data_frame=pd.DataFrame(results)
    data_frame.set_index('zipcode',inplace=True)
#     data_frame.to_csv(file_name)
    for year in years[1:]:
        time.sleep(1)
        census_url="https://api.census.gov/data/{}/zbp?".format(year)
        final_url=census_url+get+"&key="+key
        results={}
        business_response=requests.get(final_url).json()
        print(business_response[0])
        estab_name='ESTAB_{}'.format(year)
        results[estab_name]=[i[0] for i in business_response]
        results['zipcode']=[i[4] for i in business_response] 
        df_new=pd.DataFrame(results)
        df_new.set_index('zipcode',inplace=True)
        data_frame = data_frame.join(df_new,how='inner')
    return data_frame     
    

In [4]:
years_list = [2012,2013,2014,2015,2016]
df_estab = get_res(years_list)

['ESTAB', 'GEO_TTL', 'YEAR', 'NAICS2012', 'zipcode']
['ESTAB', 'GEO_TTL', 'YEAR', 'NAICS2012', 'zipcode']
['ESTAB', 'GEO_TTL', 'YEAR', 'NAICS2012', 'zipcode']
['ESTAB', 'GEO_TTL', 'YEAR', 'NAICS2012', 'zipcode']
['ESTAB', 'GEO_TTL', 'YEAR', 'NAICS2012', 'zipcode']


In [5]:
df_estab = df_estab[1:]
df_estab.head()

,ESTAB_2012,ESTAB_2013,ESTAB_2014,ESTAB_2015,ESTAB_2016
zipcode,,,,,
01001,35,36,36,37,33
01002,63,57,63,59,58
01004,1,3,1,1,2
01005,8,10,9,10,10
01007,15,17,17,18,18


In [6]:
df_estab.to_csv('zbp_estab_2012_2016.csv')

 - Restaurants are business with the NAICS2012 code of  7225. Link for the code explanation is here:
 https://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart_code=72&search=2012%20NAICS%20Search
 
 - data is saved as zbp_establ_2012_2016.csv
 

## Get demo,house,social_economic data from ACS5 profile dataset

In [30]:
#Initial Features list 
demographic_variables = ['DP04_0133E','DP04_0083E','DP02_0092E','DP03_0122E','DP02_0001E','DP02_0116E','DP02_0111E',
                        'DP04_0004E','DP04_0132E','DP04_0124E','DP02_0095E','DP05_0026E','DP02_0058E',
                        'DP05_0064E','DP05_0059E','DP03_0044E','DP03_0115E','DP04_0005E','DP02_0090E',
                        'DP03_0040E','DP04_0033E','DP04_0086E','DP02_0060E','DP05_0062E',
                        'DP02_0010E','DP02_0016E','DP04_0087E','DP04_0034E','DP04_0002E','DP04_0038E',
                        'DP04_0035E','DP04_0081E','DP02_0008E','DP04_0109E','DP03_0052E','DP05_0029E',
                        'DP03_0054E','DP03_0092E','DP05_0074E','DP03_0051E','DP02_0121E','DP05_0027E',
                        'DP04_0073E','DP02_0020E','DP03_0014E','DP04_0003E','DP02_0056E','DP02_0098E',
                        'DP02_0120E','DP03_0117E','DP02_0069E','DP04_0082E','DP03_0083E','DP03_0046E',
                        'DP05_0060E','DP05_0031E','DP04_0030E','DP04_0110E','DP04_0140E',
                        'DP02_0021E','DP02_0012E','DP05_0033E','DP05_0002E','DP03_0043E','DP04_0070E',
                        'DP03_0027E','DP04_0041E','DP04_0036E','DP05_0079E','DP02_0128E','DP05_0025E',
                        'DP02_0068E','DP02_0112E','DP04_0042E','DP05_0003E','DP03_0028E','DP02_0127E',
                        'DP02_0009E','DP02_0119E','DP04_0072E','DP02_0050E','DP03_0050E','DP04_0084E',
                        'DP02_0115E','DP02_0051E','DP04_0080E','DP03_0114E','DP02_0093E',
                        'DP03_0091E','DP03_0004E','DP05_0063E','DP03_0057E','DP05_0071E','DP02_0129E',
                        'DP03_0095E','DP04_0037E','DP05_0077E','DP02_0017E','DP04_0100E',
                        'DP03_0116E','DP02_0004E','DP02_0011E','DP03_0096E','DP04_0001E','DP05_0038E',
                        'DP05_0032E','DP03_0119E','DP03_0090E','DP02_0002E','DP04_0045E','DP03_0055E',
                        'DP03_0013E','DP03_0099E','DP04_0008E','DP04_0139E','DP03_0093E','DP03_0112E',
                        'DP05_0073E','DP03_0110E','DP03_0048E','DP03_0058E','DP03_0007E','DP03_0049E',
                        'DP04_0088E','DP02_0099E','DP02_0064E','DP03_0097E','DP02_0118E','DP03_0015E',
                        'DP02_0054E','DP02_0117E','DP02_0122E','DP04_0032E','DP02_0015E','DP04_0075E',
                        'DP03_0005E','DP02_0052E','DP03_0056E','DP03_0082E','DP05_0034E','DP03_0113E',
                        'DP03_0094E','DP02_0059E','DP02_0013E','DP02_0125E','DP04_0043E','DP03_0006E',
                        'DP04_0077E','DP04_0047E','DP03_0029E','DP05_0035E','DP04_0076E','DP03_0047E',
                        'DP02_0062E','DP02_0126E','DP04_0107E','DP03_0062E','DP03_0012E','DP02_0053E',
                        'DP02_0055E','DP02_0014E','DP02_0057E','DP03_0059E','DP05_0076E','DP03_0009E',
                        'DP02_0094E','DP04_0108E','DP05_0039E','DP03_0111E','DP05_0028E','DP05_0024E',
                        'DP03_0010E','DP04_0085E','DP02_0114E','DP05_0061E','DP02_0123E','DP04_0071E',
                        'DP02_0113E','DP02_0019E','DP05_0078E','DP02_0065E','DP03_0098E','DP02_0091E',
                        'DP03_0118E','DP04_0046E','DP05_0001E','DP03_0025E','DP03_0011E','DP05_0030E',
                        'DP05_0037E','DP04_0089E','DP02_0063E','DP04_0007E','DP02_0130E','DP02_0124E',
                        'DP05_0075E','DP02_0096E','DP04_0074E','DP03_0120E','DP04_0044E','DP03_0031E',
                        'DP03_0053E','DP02_0110E','DP04_0111E','DP03_0030E','DP02_0018E','DP03_0001E',
                        'DP02_0003E','DP04_0031E','DP05_0017E','DP03_0045E','DP03_0041E','DP03_0008E',
                        'DP03_0002E','DP02_0066E','DP04_0049E','DP04_0040E','DP05_0070E','DP02_0061E',
                        'DP02_0097E','DP04_0006E','DP02_0005E','DP05_0072E','DP03_0086E','DP05_0036E']

In [33]:
#function to get demographic info 
def get_demo(year):
    variable = demographic_variables[0]
    time.sleep(1)
    if year>=2015:
        acs5_root = 'https://api.census.gov/data/{}/acs/acs5/profile?'.format(year) 
    else:
        acs5_root ='https://api.census.gov/data/{}/acs5/profile?'.format(year)
    #     'get=DP03_0001E,DP03_0001PE,NAME&for=zip%20code%20tabulation%20area:78759&key='
    acs_get='get={}&for=zip%20code%20tabulation%20area:*&key='.format(variable)
    key='38f084f3e7ae058abf3d8155617fface08b2f507'
    acs5_final_url=acs5_root+acs_get+key
    #     print(acs5_final_url)
    response = requests.get(acs5_final_url).json()
    #     print(response)
    df = pd.DataFrame(response)
    df.columns = response[0]
    df.set_index('zip code tabulation area',inplace=True)
    
    for i in demographic_variables[1:]:
        time.sleep(1)
        acs_get='get={}&for=zip%20code%20tabulation%20area:*&key='.format(i)
        key='38f084f3e7ae058abf3d8155617fface08b2f507'
        acs5_final_url=acs5_root+acs_get+key
    #     print(acs5_final_url)
        response = requests.get(acs5_final_url)
        try:
            response = response.json()
#             print(response[0])
            df_new = pd.DataFrame(response)
            df_new.columns = response[0]
            df_new.set_index('zip code tabulation area',inplace=True)
            df=df.join(df_new,how='inner')
        except:
            print('JSONDecodeError no feature in this file')
            
    return df

In [ ]:
years=[2012,2014,2016]
# years = [2014,2016]
for year in years:
    year_df=get_demo(year)
    year_df = year_df[1:]
    print(year_df.shape)
    year_df.to_csv('{}_demographic_zip.csv'.format(year))

- Demographic,house,social-economical data are from ACS5/profile. The API link to census dataset is https://api.census.gov/data.html


- An easier way to navigate through these variables to see what fits our purpose is to go to the advance search link here https://factfinder.census.gov/faces/nav/jsf/pages/searchresults.xhtml?refresh=t 
 Search Table name DP02, DP03,DP04,DP05 one by one and find relevant variable names at the link above at corresponding ACS5 file


- I found around 40 variables that I see relevant but there are around 3000 variables you can choose. Feel free to choose any if you see appropriate.Just need to add the varaible at the demographic_variables list and run the code below to generate a new csv data.


# Data  Processing

In [97]:
# take a look at one year's demo data for example
demo_2012 = pd.read_csv('2012_demographic_zip.csv')
demo_2014 = pd.read_csv('2014_demographic_zip.csv')
demo_2016 = pd.read_csv('2016_demographic_zip.csv')

/Users/yizhiyin/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (80,198) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yizhiyin/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (195) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [98]:
print('2012 shape is {}\n'.format(demo_2012.shape))
print('2014 shape is {}\n'.format(demo_2014.shape))
print('2016 shape is {}\n'.format(demo_2016.shape))


2012 shape is (33120, 226)

2014 shape is (32989, 221)

2016 shape is (33120, 226)



** Eventually we need to calculate the difference between demo_2014 and demo_2012, and demo_2016 to demo_2014 for each feature, it seems there are 131 and 5 columns missing in demo_2014. Those missing rows all have 3 digit zipcodes (Puerto Rico), while the missing columns are features not included in 2014 ACS5. We need to drop these 131 and the extra 5 features from demo_2012.  Note our recommendation system does not include Puerto Rico due to absence of data.**

### delete 131 missing rows 

In [ ]:
#before we drop those 131 rows, 
#take a closer look at if there are other na rows in other feature which we do not want to drop
demo_2012.isnull().sum()

In [100]:
#ok, it seems there are only 131 rows missing DP02 group information, we can dropna now
demo_2012 = demo_2012.dropna(axis=0)
demo_2012.shape
#ok, now we dropped these 131 rows

(32989, 226)

In [ ]:
demo_2016.isnull().sum()

In [102]:
#ok, it seems there are only 131 rows missing DP02 group information, we can dropna now
demo_2016 = demo_2016.dropna(axis=0)
demo_2016.shape[0] == demo_2014.shape[0]


True

### Convert 'object' data to numeric data
 - all the features we picked should be numeric but sometimes they come as 'object' type

In [103]:
def to_numeric(file_name):
    object_mask = file_name.dtypes=='object'
    object_features = file_name.dtypes[object_mask].index
    for i in object_features:
        file_name[i] = pd.to_numeric(file_name[i],'float')
    

In [104]:
#check out data types
print(set(demo_2012.dtypes))
print(set(demo_2014.dtypes))
print(set(demo_2016.dtypes))

{dtype('float64'), dtype('O'), dtype('int64')}
{dtype('O'), dtype('int64')}
{dtype('float64'), dtype('int64')}


In [105]:
to_numeric(demo_2012)
to_numeric(demo_2014)
to_numeric(demo_2016)
#check out data types
print(set(demo_2012.dtypes))
print(set(demo_2014.dtypes))
print(set(demo_2016.dtypes))

{dtype('float64'), dtype('int64')}
{dtype('float64'), dtype('int64')}
{dtype('float64'), dtype('int64')}


# API  Get shared variables label 

In [106]:
#Now conslidate all common variables of all years after processing in one table and get the label name
demographic_variables=list(set(list(demo_2012.columns))&set(list(demo_2014.columns)))
demographic_variables.remove('zip code tabulation area')
variables_url= 'https://api.census.gov/data/2012/acs5/profile/variables.json'
demographic_labels=[]
response = requests.get(variables_url).json()
for i in demographic_variables:
    demographic_labels.append(response['variables'][i]['label'])
variable_pd = pd.DataFrame({'variable':demographic_variables,'label':demographic_labels})

variable_pd.to_csv('variable_label.csv')


## Add number of restaurants of each year into the dataset

** As I found out later, there are plenty of zipcodes are mail box zipcodes which cause lots of trouble for processing later. One way to exclude those zipcodes is to add the number of establishments as a column and filter out those ones with null.**

In [107]:
df_estab = pd.read_csv('zbp_estab_2012_2016.csv')
df_estab.head()

,zipcode,ESTAB_2012,ESTAB_2013,ESTAB_2014,ESTAB_2015,ESTAB_2016
0,1001,35,36,36,37,33
1,1002,63,57,63,59,58
2,1004,1,3,1,1,2
3,1005,8,10,9,10,10
4,1007,15,17,17,18,18


In [108]:
len(df_estab)

22835

** If we compare zipcode 1003 in the demo_2012 and df_estab, we can see it has a total household(DP02_0001E) of 17 and no business established. I think our client will not likely consider to open any restaurant in an area like that. I am going to filter these one out by doing an inner_join of these tables.**

In [109]:

demo_2012_estab = demo_2012.set_index('zip code tabulation area').join\
                (df_estab.set_index('zipcode').ESTAB_2012,how='inner')
demo_2012_estab.head()  

,DP04_0133E,DP04_0083E,DP02_0092E,DP03_0122E,DP02_0001E,DP02_0116E,DP02_0111E,DP04_0004E,DP04_0132E,DP04_0124E,...,DP04_0040E,DP05_0070E,DP02_0061E,DP02_0097E,DP04_0006E,DP02_0005E,DP05_0072E,DP03_0086E,DP05_0036E,ESTAB_2012
1001,67,1398,1760.0,NaN,7018.0,1574.0,14315.0,1.6,848.0,1710,...,2671,124,4301.0,195.0,7418,1143.0,15761,72258.0,0,35
1002,176,312,4519.0,NaN,9260.0,1470.0,22668.0,3.6,1092.0,4480,...,2805,1171,1473.0,824.0,9991,1364.0,21022,99963.0,0,63
1005,29,434,128.0,NaN,1819.0,144.0,4617.0,2.0,775.0,188,...,381,6,1264.0,31.0,2075,475.0,4979,93279.0,0,8
1007,55,439,786.0,NaN,5624.0,574.0,12627.0,1.1,895.0,1161,...,1511,163,2287.0,154.0,5709,1499.0,13444,94537.0,0,15
1008,0,115,41.0,NaN,461.0,44.0,1042.0,2.2,870.0,7,...,90,3,353.0,10.0,549,124.0,1129,80789.0,0,5


In [110]:
#save the processed data to process_data folder
demo_2012_estab.to_csv('process_data/processed_2012_demographic_zip.csv')

In [111]:
#do the same thing for demo_2014
demo_2014.head()

,zip code tabulation area,DP04_0133E,DP04_0083E,DP02_0092E,DP02_0001E,DP02_0116E,DP02_0111E,DP04_0004E,DP04_0132E,DP04_0124E,...,DP04_0049E,DP04_0040E,DP05_0070E,DP02_0061E,DP02_0097E,DP04_0006E,DP02_0005E,DP05_0072E,DP03_0086E,DP05_0036E
0,1001,49,1510,1646,7000,1189,13910,0.0,895.0,1963,...,7000,2701,154,4157,529,7372,1193,15165,74668.0,0
1,1002,171,266,5022,9494,1420,24015,2.6,1117.0,4881,...,9494,3028,1457,1442,642,10292,1382,21934,100486.0,0
2,1003,0,0,1208,16,264,10237,NaN,NaN,16,...,16,0,246,0,261,16,12,8189,NaN,0
3,1005,30,426,112,1835,116,4698,2.6,906.0,277,...,1835,671,5,1314,45,2179,349,4929,83676.0,0
4,1007,60,564,563,5730,368,13034,0.8,876.0,1153,...,5730,1645,90,2534,149,5886,1473,13659,89717.0,0


In [112]:
demo_2014_estab = demo_2014.set_index('zip code tabulation area').join\
                (df_estab.set_index('zipcode').ESTAB_2014,how='inner')
demo_2014_estab.head()   

,DP04_0133E,DP04_0083E,DP02_0092E,DP02_0001E,DP02_0116E,DP02_0111E,DP04_0004E,DP04_0132E,DP04_0124E,DP02_0095E,...,DP04_0040E,DP05_0070E,DP02_0061E,DP02_0097E,DP04_0006E,DP02_0005E,DP05_0072E,DP03_0086E,DP05_0036E,ESTAB_2014
1001,49,1510,1646,7000,1189,13910,0.0,895.0,1963,790,...,2701,154,4157,529,7372,1193,15165,74668.0,0,36
1002,171,266,5022,9494,1420,24015,2.6,1117.0,4881,3503,...,3028,1457,1442,642,10292,1382,21934,100486.0,0,63
1005,30,426,112,1835,116,4698,2.6,906.0,277,70,...,671,5,1314,45,2179,349,4929,83676.0,0,9
1007,60,564,563,5730,368,13034,0.8,876.0,1153,260,...,1645,90,2534,149,5886,1473,13659,89717.0,0,17
1008,0,92,50,493,29,1135,0.0,821.0,29,18,...,112,3,290,7,583,113,1195,78750.0,0,4


In [113]:
#save the processed data to process_data folder
demo_2014_estab.to_csv('process_data/processed_2014_demographic_zip.csv')

In [114]:
demo_2016_estab = demo_2016.set_index('zip code tabulation area').join\
                (df_estab.set_index('zipcode').ESTAB_2016,how='inner')
demo_2016_estab.head()   

,DP04_0133E,DP04_0083E,DP02_0092E,DP03_0122E,DP02_0001E,DP02_0116E,DP02_0111E,DP04_0004E,DP04_0132E,DP04_0124E,...,DP04_0040E,DP05_0070E,DP02_0061E,DP02_0097E,DP04_0006E,DP02_0005E,DP05_0072E,DP03_0086E,DP05_0036E,ESTAB_2016
1001,151,636,1684.0,-888888888,7190.0,1523.0,13810.0,0.0,0,128,...,852,102,3907.0,776.0,7482,1017.0,15079,82512,0,33
1002,21,51,4940.0,-888888888,9561.0,1288.0,24140.0,2.1,66,203,...,1431,992,1678.0,443.0,10498,1352.0,22082,94489,0,58
1005,0,277,84.0,-888888888,1840.0,99.0,4964.0,2.7,0,47,...,268,1,1412.0,33.0,2189,344.0,5008,99127,0,10
1007,0,175,762.0,-888888888,5611.0,538.0,13006.0,0.7,0,65,...,303,91,2251.0,187.0,5903,1462.0,13601,92100,0,18
1008,0,41,36.0,-888888888,530.0,31.0,1122.0,1.8,0,26,...,28,0,295.0,6.0,649,89.0,1178,72000,0,5


In [115]:
#save the processed data to process_data folder
demo_2016_estab.to_csv('process_data/processed_2016_demographic_zip.csv')

In [116]:
len(demo_2014_estab) == len(demo_2012_estab)==len(demo_2016_estab)

True

**All right, now all data by year has the same number of rows and columns. Now let's do calculation for data we are going to train the model.**

## Create analytic table
- 2014_demo_feature - 2012_demo_feature
- (2014_demo_feature - 2012_demo_feature)/2012_demo_feature

In [14]:
demo_2012_estab = pd.read_csv('process_data/processed_2012_demographic_zip.csv')
demo_2014_estab = pd.read_csv('process_data/processed_2014_demographic_zip.csv')
demo_2016_estab = pd.read_csv('process_data/processed_2016_demographic_zip.csv')

In [15]:
len(demo_2014_estab) == len(demo_2012_estab)==len(demo_2016_estab)

True

In [16]:
demo_2014_estab.head()

,Unnamed: 0,DP04_0133E,DP04_0083E,DP02_0092E,DP02_0001E,DP02_0116E,DP02_0111E,DP04_0004E,DP04_0132E,DP04_0124E,...,DP04_0040E,DP05_0070E,DP02_0061E,DP02_0097E,DP04_0006E,DP02_0005E,DP05_0072E,DP03_0086E,DP05_0036E,ESTAB_2014
0,1001,49,1510,1646,7000,1189,13910,0.0,895.0,1963,...,2701,154,4157,529,7372,1193,15165,74668.0,0,36
1,1002,171,266,5022,9494,1420,24015,2.6,1117.0,4881,...,3028,1457,1442,642,10292,1382,21934,100486.0,0,63
2,1005,30,426,112,1835,116,4698,2.6,906.0,277,...,671,5,1314,45,2179,349,4929,83676.0,0,9
3,1007,60,564,563,5730,368,13034,0.8,876.0,1153,...,1645,90,2534,149,5886,1473,13659,89717.0,0,17
4,1008,0,92,50,493,29,1135,0.0,821.0,29,...,112,3,290,7,583,113,1195,78750.0,0,4


In [17]:
demo_2012_estab.rename(columns={'Unnamed: 0': 'zipcode'}, inplace=True)

demo_2014_estab.rename(columns={'Unnamed: 0': 'zipcode'}, inplace=True)

demo_2016_estab.rename(columns={'Unnamed: 0': 'zipcode'}, inplace=True)


In [20]:
demo_2012_estab.set_index('zipcode',inplace=True)
demo_2014_estab.set_index('zipcode',inplace=True)
demo_2016_estab.set_index('zipcode',inplace=True)


In [117]:
demographic_variables = pd.read_csv('variable_label.csv').variable.values

pd_series = [demo_2014_estab[i]-demo_2012_estab[i] for i in demographic_variables] +\
            [demo_2012_estab.ESTAB_2012,demo_2014_estab.ESTAB_2014,demo_2016_estab.ESTAB_2016]
delta_demo = pd.concat(pd_series, axis=1)
delta_demo.head()

,DP05_0001E,DP04_0002E,DP03_0005E,DP03_0094E,DP03_0043E,DP03_0006E,DP02_0097E,DP03_0090E,DP05_0074E,DP02_0127E,...,DP03_0053E,DP04_0004E,DP05_0028E,DP05_0071E,DP02_0128E,DP02_0052E,DP05_0078E,ESTAB_2012,ESTAB_2014,ESTAB_2016
1001,-239,-18,115,283.0,8,11,334.0,-10140.0,10,34.0,...,-16,-1.6,-239,-441,303.0,-502.0,-13,35,36,33
1002,1126,234,-99,14.0,347,-17,-182.0,-2793.0,19,-183.0,...,-208,-1.0,1126,1224,6.0,879.0,-1,63,63,58
1005,-7,16,-9,-3105.0,-82,-13,14.0,-7039.0,-2,13.0,...,10,0.6,-7,-16,-391.0,117.0,1,8,9,10
1007,181,106,-26,1462.0,57,0,-5.0,-1579.0,0,40.0,...,-2,-0.3,181,215,108.0,241.0,13,15,17,18
1008,52,32,23,4278.0,17,3,-3.0,13274.0,0,-5.0,...,4,-2.2,52,59,-17.0,1.0,0,5,4,5


In [118]:
#save it
delta_demo.to_csv('process_data/analytic_zip.csv')

In [23]:
# create a percentage table
demographic_variables = pd.read_csv('variable_label.csv').variable.values

pd_series = [(demo_2014_estab[i]-demo_2012_estab[i])*100/demo_2012_estab[i] for i in demographic_variables] +\
            [demo_2012_estab.ESTAB_2012,demo_2014_estab.ESTAB_2014,demo_2016_estab.ESTAB_2016]
delta_demo = pd.concat(pd_series, axis=1)
delta_demo.head()

,DP05_0001E,DP04_0002E,DP03_0005E,DP03_0094E,DP03_0043E,DP03_0006E,DP02_0097E,DP03_0090E,DP05_0074E,DP02_0127E,...,DP03_0053E,DP04_0004E,DP05_0028E,DP05_0071E,DP02_0128E,DP02_0052E,DP05_0078E,ESTAB_2012,ESTAB_2014,ESTAB_2016
zipcode,,,,,,,,,,,,,,,,,,,,,
1001,-1.375144,-0.256483,18.110236,0.644882,0.974421,42.307692,171.282051,-23.858824,66.666667,62.962963,...,-7.766990,-100.000000,-1.375144,-2.661597,17.099323,-12.073112,-8.904110,35,36,33
1002,3.920886,2.526998,-7.432432,0.029717,25.608856,-100.000000,-22.087379,-9.035618,26.027397,-40.486726,...,-26.908150,-27.777778,3.920886,4.641990,0.166574,5.575996,-0.131234,63,63,58
1005,-0.136719,0.879604,-3.114187,-6.515444,-36.607143,-100.000000,45.161290,-19.849976,-22.222222,7.103825,...,27.777778,30.000000,-0.136719,-0.316331,-31.481481,9.543230,inf,8,9,10
1007,1.240321,1.884780,-4.569420,3.203401,10.555556,NaN,-3.246753,-4.503579,NaN,44.444444,...,-1.111111,-27.272727,1.240321,1.516006,5.678233,6.122967,4.347826,15,17,18
1008,4.482759,6.941432,82.142857,9.929671,34.693878,75.000000,-30.000000,26.771272,NaN,-17.241379,...,40.000000,-100.000000,4.482759,5.148342,-6.049822,0.390625,NaN,5,4,5


In [24]:
delta_demo.to_csv('process_data/rate_analytic_zip.csv')

## Now I can move on to explore features. I am going to start to work on another jupyter notebook.
